## Import

In [ ]:
from ALLCools.clustering import *
from ALLCools.mcds import MCDS
from wmb import brain, aibs, broad, mm10

import pandas as pd
import anndata

import matplotlib.pyplot as plt
from ALLCools.plot import *

import scanpy as sc

In [ ]:
n_cell = 5
dataset = 'AIBS_SMART'

## Select cells

In [ ]:
cells = pd.read_csv('rna_cells.txt', index_col=0, header=None).index
cells.name = 'cell'
cells.size

## Get adata with basic feature selection

In [ ]:
if dataset == 'AIBS_SMART':
    mcds_path = aibs.AIBS_SMART_ZARR_PATH
    gene_map = aibs.get_smart_gene_map()
elif dataset == 'AIBS_TENX':
    mcds_path = aibs.AIBS_TENX_ZARR_PATH
    gene_map = aibs.get_tenx_gene_map()
else:
    mcds_path = broad.BROAD_TENX_ZARR_PATH
    gene_map = broad.get_tenx_gene_map()

In [ ]:
rna_ds = MCDS.open(mcds_path, use_obs=cells, var_dim='gene')

In [ ]:
rna_ds

In [ ]:
merfish = anndata.read_h5ad('./adata/merfish_input.h5ad')

In [ ]:
#gene_id_not_nan = ~rna_ds.get_index('gene').map(gene_map).isna()
is_merfish_probe = rna_ds.get_index('gene').isin(merfish.var_names)
is_merfish_probe.sum()

In [ ]:
adata = rna_ds.get_count_adata(da_name='gene_da', use_vars=is_merfish_probe)

In [ ]:
adata

In [ ]:
# cleanup gene metadata of RNA
assert adata.var_names.duplicated().sum() == 0

# gene_meta = mm10.get_gene_metadata()
# for col in ['chrom', 'start', 'end']:
#     adata.var[col] = gene_meta[col]

## Basic Feature Filtering

In [ ]:
# no need to do this, already did in mC

# chroms = [
#     'chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9',
#     'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17',
#     'chr18', 'chr19'
# ]
#
# remove_chromosomes(adata, include_chromosomes=chroms)
# remove_black_list_region(adata, black_list_path=mm10.ENCODE_BLACKLIST_PATH)

In [ ]:
if adata.n_obs > n_cell:
    filter_regions(adata, n_cell=n_cell, zscore_abs_cutoff=None)

In [ ]:
sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)
sc.pp.scale(adata)

In [ ]:
adata.write_h5ad('./adata/rna_input.h5ad')

In [ ]:
adata